In [1]:
#!pip install Census
#!pip install us 
#!pip install plotly==4.1.0

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import us
from census import Census
from us import states

In [2]:
# import api key
from config import api_key
#api_key


In [3]:
# male and female data from census
c18 = Census(api_key,year=2018)


In [4]:
# function_1 to merge to dataframe
def func_merge_df(df1, df2, column):
    mf_df = pd.merge(df1, df2, on=column, how='inner')
    return mf_df

#function_1 to pull data and merge using function_1
def func_getdata(key1, key2, jobtype):
    male_data= []
    female_data = []
    state_dict = us.states.mapping('fips', 'name')
    
    male_data=c18.acs1.get(key1, {'for': 'state:*'})
    female_data=c18.acs1.get(key2, {'for': 'state:*'})
    m_pd = pd.DataFrame(male_data)
    f_pd = pd.DataFrame(female_data)
    # Merge the dataframe
    mf_df = func_merge_df(df1=f_pd, df2=m_pd, column='state')
    mf_df=mf_df.fillna(0)
    # Get the total 
    mf_df["State Name"] = ''
    mf_df["Type"] = ''
    # Get the State name
    for index, row in mf_df.iterrows():
        mf_df.loc[index,'State Name'] = state_dict[mf_df.loc[index,'state']]
        mf_df.loc[index,'Type'] = jobtype
    return mf_df

In [5]:
# excute func_1and func_2 to obtain data clean the data and export to a csv file

cnt = 0
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_170E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)

for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 
    if cnt !=0:
        cnt = cnt+1

# First time for the header
    if cnt == 0:
        rs_data.to_csv("job_data_output.csv", encoding="utf-8", index=False)
        cnt = cnt + 1 

    if cnt > 1:
        with open('job_data_output.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

Job Completed
